In [ ]:
import numpy as np 
import pandas as pd 
import os

In [ ]:
columns_names = ['Reports']
depositions_and_reports = pd.read_csv('/kaggle/input/depositionsandreports/InternetOfThingsCaseStudy_report.csv',sep="\r", names=columns_names)
crime_scene_reports = pd.read_csv('/kaggle/input/depositionsandreports/crime_scene_report.csv',sep="\r", names=columns_names)

In [ ]:
from transformers import AutoTokenizer
import transformers
import torch

model = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model)
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
def get_output(messages):
    prompt = pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
    )

    outputs = pipeline(
        prompt,
        max_new_tokens=1024,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )
    print(outputs[0]["generated_text"][len(prompt):])


def chat_with_llama(dataset_column, object_location=False):
    messages = []
    
    if object_location:
        messages.append({"role": "system", "content": "Translate the following sentence in Answer Set Programming facts. When answering, use predicates 'east(object1, object2)', 'west(object1, object2)', 'south(object1, object2)', 'north(object1, object2)', 'superior(object1, object2)', 'same_level(object1, object2)'. Produce as output just the translation and not the explanation. Example of sentence: 'The body was over the ground and, at the right of it, there was a knife'. Example of translation: superior(body, ground). east(body, knife). same_level(body, knife).  Sentence:"})
    else:
        messages.append({"role": "system", "content": "Translate the following sentence in Answer Set Programming facts. When answering, use predicates 'timestamp(year-month-day_hour:minute:second)', 'event(what_happened, year-month-day_hour , where)'. Produce as output just the translation and not the explanation. Example of sentence: '23:00 2024-02-09 The man laying on the sofa called his wife and wishes her a good birthday'. Example of translation: timestamp(2024-02-09_23:00).  event(the_man_called_his_wife, 2024-02-09_23:00, apartment). Sentence:"})
    
    for report in dataset_column:
        messages.append({"role": "user", "content": f"{report}"})
        get_output(messages)
        messages.pop()

In [ ]:
terminators = [
    pipeline.tokenizer.eos_token_id,
    pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
    
chat_with_llama(depositions_and_reports['Reports'])
chat_with_llama(crime_scene_reports['Reports'], True)